In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, TimeSeriesSplit
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
root = '/kaggle/input/store-sales-time-series-forecasting/'

train_data = pd.read_csv(os.path.join(root, 'train.csv'), index_col=0)
test_data_orig = pd.read_csv(os.path.join(root, 'test.csv'), index_col=0)
oil_df = pd.read_csv(os.path.join(root, 'oil.csv'))
samp_subm = pd.read_csv(os.path.join(root, 'sample_submission.csv'))
holiday_event_df = pd.read_csv(os.path.join(root, 'holidays_events.csv'))
stores_df =  pd.read_csv(os.path.join(root, 'stores.csv'))
transactions_df = pd.read_csv(os.path.join(root, 'transactions.csv'))

In [ ]:
# Merge
train_data = train_data.merge(stores_df, on = 'store_nbr') 
train_data = train_data.merge(oil_df, on ='date', how = 'left')
holiday_event_df = holiday_event_df.rename(columns = {'type': 'holiday'})
holiday_event_df['holiday'] = holiday_event_df['holiday'].apply(lambda x: 1 if x == 'Holiday' else 0)
holiday_event_df['holiday'].fillna(0, inplace=True)

train_data = train_data.merge(holiday_event_df, on = 'date', how = 'left')
test_data = test_data_orig.merge(holiday_event_df, on = 'date', how = 'left')

train_data['holiday'].fillna(0, inplace=True)
test_data['holiday'].fillna(0, inplace=True)

test_data['sales'] = 0.0
test_data_orig['sales'] = 0.0

In [ ]:
train_data['holiday'].unique()

In [ ]:
req_columns = ['sales', 'onpromotion', 'holiday']
req_columns_agg = {'sales':'mean', 'onpromotion':'mean', 'holiday': 'max'}
# req_columns = {'sales':'mean'}
train_data = train_data.groupby(['date']).agg(req_columns_agg)
test_data = test_data.groupby(['date']).agg(req_columns_agg)

train_data = train_data[req_columns]
test_data = test_data[req_columns]

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        # label = data[i+seq_length]
        sequences.append(seq)
    # print(sequences)
    return np.array(sequences)
    
def get_ytrain(train_data, seq_length):
    # Extract the 'sales' column from the train data
    y_train = train_data['sales'].values.reshape(-1, 1)
    print(len(y_train))
    return y_train[seq_length:]

def get_xtrain_xtest(train_data, test_data, req_columns, seq_length=30):
    # Normalize the data
    # shift on promotion and holiday by one?
    # test_data_scaled = None
    # if add_cols:
    #     scaler = MinMaxScaler()
    #     scaler.fit_transform(train_data[add_cols])
    #     test_data_scaled = scaler.transform(test_data[add_cols])
    
    scaler = MinMaxScaler()
    train_data_scaled = scaler.fit_transform(train_data[req_columns])
    # test_data_scaled = scaler.transform(test_data[req_columns])
    
    train_data_scaled = create_sequences(train_data_scaled, seq_length)

    # return train_data_scaled, test_data_scaled, scaler
    return train_data_scaled, scaler

def get_next_seq(n, new_sale_val, train_data_scaled, test_data, scaler, seq_length):
    # create new instance in test_data
    new_instance = test_data.values[n]
    new_instance[0] = new_sale_val
    new_instance = np.array([new_instance])
    
    # transform the data to be added in xtrain
    print(f'New instance before transform: {new_instance}')
    new_instance_scaled =  scaler.transform(new_instance)
    print(f'New instance after transform: {new_instance_scaled}')
    
    # insert new value to x_train
    new_sequance = train_data_scaled[-1][1:]
    new_sequance = np.vstack([new_sequance, new_instance_scaled[0]])
    train_data_scaled = np.concatenate([train_data_scaled, new_sequance[np.newaxis, :, :]])
    # np.insert(train_data_scaled, len(train_data_scaled), new_instance_scaled[0], axis=0)
    
    # get new sequance. last seq_length items in x_train
    new_sequance = train_data_scaled[-1]
    new_sequance = np.array([new_sequance])    
    # print(new_sequance)
    
    return train_data_scaled, new_sequance

def get_predictions(model, test_data_orig, test_data, x_train, y_train, scaler, seq_length):
    new_sale_val = y_train[-1]
    for i in range(len(test_data)):
        x_train, new_sequance = get_next_seq(i, new_sale_val, x_train, test_data, scaler, seq_length)

        # predict
        sales_predicted = model.predict(new_sequance)
        print(sales_predicted)
        new_sale_val = sales_predicted[-1][0]
        date = test_data.index[i]
        print(f'Predict value for {date} : {new_sale_val}')
        
        # add predicted value to test_data
        test_data.iloc[i]['sales'] = new_sale_val
    
    return test_data

In [ ]:
seq_length=30
x_train, scaler = get_xtrain_xtest(train_data, test_data, req_columns, seq_length=seq_length)
y_train = get_ytrain(train_data, seq_length)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
def validate_model(model, x_train, y_train, fit_args, n_splits=5):
    # Initialize lists to store evaluation metrics
    mse_scores = []
    mae_scores = []
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    i = 0
    # Perform k-fold cross-validation
    for train_index, val_index in tscv.split(x_train):
        # Split the data into training and validation sets
#         print(train_index)
        X_train, X_val = x_train[train_index], x_train[val_index]
        Y_train, Y_val = y_train[train_index], y_train[val_index]
        
        # Train the model
        model.fit(X_train, Y_train, **fit_args)

        # Make predictions on the validation set
        val_predictions = model.predict(X_val)

        # Evaluate the model
        mse = mean_squared_error(Y_val, val_predictions)
        print(f'mse of fold: {mse}')
        mae = mean_absolute_error(Y_val, val_predictions)
        print(f'mse of fold: {mae}')

        mse_scores.append(mse)
        mae_scores.append(mae)
        i += 1
    # Display the average performance metrics across folds
    print(f'Average Mean Squared Error: {np.mean(mse_scores)}')
    print(f'Average Mean Absolute Error: {np.mean(mae_scores)}')

In [ ]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, x_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

fit_args = {'epochs': 500, 'batch_size': 32, 'verbose': 0}

In [ ]:
validate_model(model, x_train, y_train, fit_args, n_splits=3)

In [ ]:
y_train.shape

In [ ]:
# model.fit(X_train[:, :-1], X_train[:, -1], epochs=50, batch_size=32)

model.fit(x_train, y_train, **fit_args)

In [ ]:
test_data = get_predictions(model, test_data_orig, test_data, x_train, y_train, scaler, seq_length)

In [ ]:
submission_df = test_data_orig.reset_index()
submission_df = submission_df.drop(columns=['sales'])
submission_df = pd.merge(submission_df, test_data, on='date', how='left')
submission_df

In [ ]:
submission_df = submission_df[['id', 'sales']]
submission_df

In [ ]:
submission_df.to_csv('submission.csv', index=False)

In [ ]:
samp_subm

In [ ]:
df = pd.read_csv('/kaggle/working/submission.csv')
df